In [43]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit import Parameter
import numpy as np
from scipy.optimize import minimize

edges = [(0,1), (0,2), (1,2), (1,3)]
n_qubits = 4

In [44]:
def build_qaoa_circuit(edges, n_qubits):
    qc = QuantumCircuit(n_qubits)
    
    # Parameters
    gamma = Parameter('γ')
    beta = Parameter('β')
    
    # Initial state |+>^n
    for i in range(n_qubits):
        qc.h(i)
    
    # Cost unitary
    for i, j in edges:
        qc.cx(i, j)
        qc.rz(2 * gamma, j)
        qc.cx(i, j)
    
    # Mixer unitary
    for i in range(n_qubits):
        qc.rx(2 * beta, i)
    
    qc.measure_all()
    
    return qc, gamma, beta

In [45]:
qc, gamma, beta = build_qaoa_circuit(edges, n_qubits)
qc.draw()

┌───┐                                          ┌─────────┐           »
   q_0: ┤ H ├──■───────────────■────■───────────────■──┤ Rx(2*β) ├───────────»
        ├───┤┌─┴─┐┌─────────┐┌─┴─┐  │               │  └─────────┘           »
   q_1: ┤ H ├┤ X ├┤ Rz(2*γ) ├┤ X ├──┼───────────────┼───────■────────────────»
        ├───┤└───┘└─────────┘└───┘┌─┴─┐┌─────────┐┌─┴─┐   ┌─┴─┐   ┌─────────┐»
   q_2: ┤ H ├─────────────────────┤ X ├┤ Rz(2*γ) ├┤ X ├───┤ X ├───┤ Rz(2*γ) ├»
        ├───┤                     └───┘└─────────┘└───┘   └───┘   └─────────┘»
   q_3: ┤ H ├────────────────────────────────────────────────────────────────»
        └───┘                                                                »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«                                              ░ ┌─┐         
«   q_0: ──────────────────────────────────────░─┤M├─────────
«                                  ┌─────────┐ ░ └╥┘┌─┐      
«   q_1: ──■────■───────────────■──┤ Rx(2*β) ├─░──╫─┤M├──────
«        ┌─┴─┐  │  ┌─────────┐  │  └─────────┘ ░  ║ └╥┘┌─┐   
«   q_2: ┤ X ├──┼──┤ Rx(2*β) ├──┼──────────────░──╫──╫─┤M├───
«        └───┘┌─┴─┐├─────────┤┌─┴─┐┌─────────┐ ░  ║  ║ └╥┘┌─┐
«   q_3: ─────┤ X ├┤ Rz(2*γ) ├┤ X ├┤ Rx(2*β) ├─░──╫──╫──╫─┤M├
«             └───┘└─────────┘└───┘└─────────┘ ░  ║  ║  ║ └╥┘
«meas: 4/═════════════════════════════════════════╩══╩══╩══╩═
«                                                 0  1  2  3

In [46]:
def cut_value(bitstring, edges):
    value = 0
    for i, j in edges:
        if bitstring[i] != bitstring[j]:
            value += 1
    return value


def expected_cut(counts, edges):
    total = 0
    shots = sum(counts.values())
    
    for bitstring, count in counts.items():
        bits = tuple(int(b) for b in bitstring[::-1])
        total += count * cut_value(bits, edges)
        
    return total / shots

In [ ]:
backend = Aer.get_backend('qasm_simulator')

def run_qaoa(gamma_val, beta_val, qc, gamma, beta, edges, shots=1024):
    # assign_parameters works across older/newer qiskit versions
    bound_qc = qc.assign_parameters({
        gamma: gamma_val,
        beta: beta_val
    }, inplace=False)
    t_qc = transpile(bound_qc, backend)
    job = backend.run(t_qc, shots=shots)
    counts = job.result().get_counts()

    return expected_cut(counts, edges)

In [48]:
# Fallbacks so this cell can run standalone even if earlier cells were skipped
try:
    edges
except NameError:
    edges = [(0,1), (0,2), (1,2), (1,3)]
try:
    n_qubits
except NameError:
    n_qubits = 4
try:
    QuantumCircuit
    Parameter
    transpile
except NameError:
    from qiskit import QuantumCircuit, transpile
    from qiskit.circuit import Parameter
try:
    build_qaoa_circuit
except NameError:
    def build_qaoa_circuit(edges, n_qubits):
        qc = QuantumCircuit(n_qubits)
        gamma = Parameter('γ')
        beta = Parameter('β')
        for i in range(n_qubits):
            qc.h(i)
        for i, j in edges:
            qc.cx(i, j)
            qc.rz(2 * gamma, j)
            qc.cx(i, j)
        for i in range(n_qubits):
            qc.rx(2 * beta, i)
        qc.measure_all()
        return qc, gamma, beta
try:
    qc, gamma, beta
except NameError:
    qc, gamma, beta = build_qaoa_circuit(edges, n_qubits)
try:
    backend
except NameError:
    from qiskit_aer import Aer
    backend = Aer.get_backend('qasm_simulator')


def objective(params):
    return -run_qaoa(params[0], params[1], qc, gamma, beta, edges)


initial_params = [0.5, 0.5]

result = minimize(
    objective,
    initial_params,
    method='COBYLA'
)

result

AttributeError: 'QuantumCircuit' object has no attribute 'bind_parameters'